In [7]:
from brat2conll import Brat2Conll

In [8]:
converter = brat2Conll()

In [11]:
df = pd.concat(converter.convert("annot"))
df

,sent_id,token,token_id,label,doc_id
0,0,Observation,0,O,ped_009
1,0,Un,1,O,ped_009
2,0,nourrisson,2,O,ped_009
3,0,de,3,O,ped_009
4,0,sexe,4,O,ped_009
...,...,...,...,...,...
54,39,–,1247,O,ped_008
0,40,vaccination,1248,O,ped_008
1,40,des,1249,O,ped_008
2,40,professionnels,1250,O,ped_008


In [18]:
df_agg = df.groupby(['doc_id','sent_id'])\
            .agg({'token':lambda x: list(x),
                  'label': lambda x : list(x)})\
            .reset_index()
df_agg

,doc_id,sent_id,token,label
0,ped_008,0,"[Histoire, clinique, L’, interrogatoire, est, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,ped_008,1,"[Nous, attribuerons, par, la, suite, cette, at...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,ped_008,2,"[L’, examen, clinique, initial, trouve, un, ét...","[O, O, O, O, O, O, disease, disease, disease, ..."
3,ped_008,3,"[L’, examen, abdominal, met, en, évidence, une...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,ped_008,4,"[Le, toucher, vaginal, est, sans, anomalie, no...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...,...
64,ped_009,23,"[Pronostic, .]","[O, O]"
65,ped_009,24,"[C’, est, une, affection, bénigne, ,, l’, évol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
66,ped_009,25,"[Mais, d’, autres, poussées, peuvent, survenir...","[O, O, O, O, O, O, O, O, O, O]"
67,ped_009,26,"[Notre, patiente, n’, a, présenté, qu’, une, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
